In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import logging
import json

# Add the src directory to sys.path
path = str(Path().resolve().parent)
if path not in sys.path:
    sys.path.insert(0, path)

print(sys.path)

# load_dotenv()
# print(str(dict(os.environ.items())))


LOGGING = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "json": {
            "()": "pythonjsonlogger.jsonlogger.JsonFormatter",
            "fmt": "%(asctime)s %(levelname)s %(name)s %(message)s",

        }
    },
}


# Set up structured logging to a file. All of the cells in this notebook use
# this logger. Find them at .data/logs.jsonl.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'sessionId': record_dict.get('session_id', None),
            'runIdd': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'functionName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)


In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

from assistant_drive import Drive, DriveConfig
from form_filler_skill.guided_conversation import GuidedConversationSkill
from openai_client import AzureOpenAIAzureIdentityAuthConfig, AzureOpenAIServiceConfig, create_client
from pydantic import HttpUrl
from skill_library import Skill
from openai_client.chat_driver import ChatDriverConfig
from skill_library import Assistant
import asyncio
from form_filler_skill.guided_conversation.definitions.acrostic_poem import definition

service_config = AzureOpenAIServiceConfig(
    auth_config=AzureOpenAIAzureIdentityAuthConfig(),
    azure_openai_endpoint=HttpUrl("https://lightspeed-team-shared-openai-eastus.openai.azure.com/"),
    azure_openai_deployment="gpt-4o",
)
language_model = create_client(service_config)


# Configure the skill and its dependencies.
path = Path(".data/assistant-1")
drive = Drive(DriveConfig(root=path))
definition.resource_constraint.quantity = 5  # type: ignore
skill = GuidedConversationSkill(name="guided_conversation", language_model=language_model, definition=definition, drive=drive.subdrive("gc1"))
skill_dependency_tree: dict[str, Skill] = {skill.name: skill}

# Define the assistant.
chat_driver_config = ChatDriverConfig(
    openai_client=language_model,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
)

assistant = Assistant(name="Alice", assistant_id="gc-assistant-123", chat_driver_config=chat_driver_config, skills=skill_dependency_tree, startup_routine="guided_conversation.conversation")
await assistant.clear()
await assistant.start()


# Function that allows user input in a non-blocking manner.
async def user_input_handler() -> None:
    while True:
        user_input = await asyncio.to_thread(input, "User: ")
        if user_input == "":
            assistant.stop()
            break
        print(f"User: {user_input}", flush=True)
        await assistant.put_message(user_input)


# Start the user input in a non-blocking way.
input_task = asyncio.create_task(user_input_handler())

# Start the assistant.
async for event in assistant.events:
    print(f"Assistant: {event.message}", flush=True)  # type: ignore
    if event.message == "Conversation complete!":
        print()
        break

await assistant.wait()
await input_task

